In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sts
import sklearn as skl

In [2]:
mrvl_df = pd.read_csv('marvel-wikia-data.csv')

In [3]:
mrvl_df.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0


0. Drop all NAN rows from APPEARANCES
1. remove page id and url slug
2. separate code name from civilian 
3. Fill in nans in GSM column as NOT_GSM
4. Fill in nans in EYE column as NOT_SPECIFIED
5. Fill in nans in Hair column as NOT_SPECIFIED
6.  Ditto for SEX
7.  Drop YEAR column
8. drop the 3 rows from ALIVE that have no data mrvl_df[mrvl_df['ALIVE'].isnull()]
8. Convert FIRST APPEARANCE to datetime 

In [4]:
mrvl_df['APPEARANCES'].unique()

array([4.043e+03, 3.360e+03, 3.061e+03, 2.961e+03, 2.258e+03, 2.255e+03,
       2.072e+03, 2.017e+03, 1.955e+03, 1.934e+03, 1.825e+03, 1.713e+03,
       1.528e+03, 1.512e+03, 1.394e+03, 1.338e+03, 1.307e+03, 1.304e+03,
       1.266e+03, 1.265e+03, 1.237e+03, 1.233e+03, 1.230e+03, 1.162e+03,
       1.161e+03, 1.137e+03, 1.120e+03, 1.107e+03, 1.050e+03, 1.047e+03,
       1.007e+03, 1.000e+03, 8.860e+02, 8.810e+02, 8.800e+02, 8.780e+02,
       8.560e+02, 8.500e+02, 7.870e+02, 7.520e+02, 7.400e+02, 7.250e+02,
       7.210e+02, 7.090e+02, 7.080e+02, 6.960e+02, 6.920e+02, 6.890e+02,
       6.860e+02, 6.570e+02, 6.410e+02, 6.360e+02, 6.180e+02, 6.120e+02,
       5.990e+02, 5.900e+02, 5.670e+02, 5.580e+02, 5.500e+02, 5.480e+02,
       5.320e+02, 5.280e+02, 5.260e+02, 5.250e+02, 5.170e+02, 5.120e+02,
       5.030e+02, 5.020e+02, 5.000e+02, 4.780e+02, 4.740e+02, 4.730e+02,
       4.710e+02, 4.640e+02, 4.480e+02, 4.460e+02, 4.330e+02, 4.290e+02,
       4.260e+02, 4.220e+02, 4.150e+02, 4.080e+02, 

In [5]:
mrvl_df.ALIVE.value_counts()

Living Characters      12608
Deceased Characters     3765
Name: ALIVE, dtype: int64

In [6]:
mrvl_df.loc[mrvl_df.GSM == 'Pansexual Characters']

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
275,2023,Julie Power (Earth-616),\/Julie_Power_(Earth-616),Secret Identity,Good Characters,Blue Eyes,Strawberry Blond Hair,Female Characters,Pansexual Characters,Living Characters,150.0,Aug-84,1984.0


In [7]:
dc_df = pd.read_csv('dc-wikia-data.csv')

In [8]:
dc_df.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,"1940, April",1940.0


In [9]:
mrvl_df.info()
dc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16376 entries, 0 to 16375
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   page_id           16376 non-null  int64  
 1   name              16376 non-null  object 
 2   urlslug           16376 non-null  object 
 3   ID                12606 non-null  object 
 4   ALIGN             13564 non-null  object 
 5   EYE               6609 non-null   object 
 6   HAIR              12112 non-null  object 
 7   SEX               15522 non-null  object 
 8   GSM               90 non-null     object 
 9   ALIVE             16373 non-null  object 
 10  APPEARANCES       15280 non-null  float64
 11  FIRST APPEARANCE  15561 non-null  object 
 12  Year              15561 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 1.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6896 entries, 0 to 6895
Data columns (total 13 columns):
 #   Column     

In [10]:
mrvl_df[mrvl_df['APPEARANCES'].isnull()]

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
15280,743309,Minister of Castile D'or (Earth-616),\/Minister_of_Castile_D%27or_(Earth-616),No Dual Identity,Neutral Characters,NaN,NaN,Male Characters,NaN,Deceased Characters,NaN,Dec-39,1939.0
15281,645438,Mr. Harris' Secretary (Earth-616),\/Mr._Harris%27_Secretary_(Earth-616),No Dual Identity,Neutral Characters,NaN,Blond Hair,Female Characters,NaN,Living Characters,NaN,Oct-39,1939.0
15282,331151,N'Jaga (Earth-616),\/N%27Jaga_(Earth-616),No Dual Identity,Bad Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,Oct-39,1939.0
15283,505986,Ertve (Earth-616),\/Ertve_(Earth-616),Secret Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,NaN,Feb-40,1940.0
15284,19657,Invisible Man (Gade) (Earth-616),\/Invisible_Man_(Gade)_(Earth-616),Secret Identity,Good Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,Apr-40,1940.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach (Earth-616),\/Ru%27ach_(Earth-616),No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16372,665474,Thane (Thanos' son) (Earth-616),\/Thane_(Thanos%27_son)_(Earth-616),No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16373,695217,Tinkerer (Skrull) (Earth-616),\/Tinkerer_(Skrull)_(Earth-616),Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16374,708811,TK421 (Spiderling) (Earth-616),\/TK421_(Spiderling)_(Earth-616),Secret Identity,Neutral Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN


0. Drop all NAN rows from APPEARANCES
1. remove page id and url slug
2. separate code name from civilian 
3. Fill in nans in GSM column as NOT_GSM
4. Fill in nans in EYE column as NOT_SPECIFIED
5. Fill in nans in Hair column as NOT_SPECIFIED
6.  Ditto for SEX
7.  Drop YEAR column
8. drop the 3 rows from ALIVE that have no data mrvl_df[mrvl_df['ALIVE'].isnull()]
8. Convert FIRST APPEARANCE to datetime 

In [11]:
mrvl_df["APPEARANCES"].isnull().unique()

array([False,  True])

In [12]:
mrvl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16376 entries, 0 to 16375
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   page_id           16376 non-null  int64  
 1   name              16376 non-null  object 
 2   urlslug           16376 non-null  object 
 3   ID                12606 non-null  object 
 4   ALIGN             13564 non-null  object 
 5   EYE               6609 non-null   object 
 6   HAIR              12112 non-null  object 
 7   SEX               15522 non-null  object 
 8   GSM               90 non-null     object 
 9   ALIVE             16373 non-null  object 
 10  APPEARANCES       15280 non-null  float64
 11  FIRST APPEARANCE  15561 non-null  object 
 12  Year              15561 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 1.6+ MB


In [13]:
testdf = mrvl_df[mrvl_df["APPEARANCES"]].drop()

KeyError: "None of [Float64Index([4043.0, 3360.0, 3061.0, 2961.0, 2258.0, 2255.0, 2072.0, 2017.0,\n              1955.0, 1934.0,\n              ...\n                 nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,\n                 nan,    nan],\n             dtype='float64', length=16376)] are in the [columns]"

In [16]:
test_df = mrvl_df[mrvl_df["APPEARANCES"] == mrvl_df["APPEARANCES"].astype(float)]

In [17]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15280 entries, 0 to 15279
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   page_id           15280 non-null  int64  
 1   name              15280 non-null  object 
 2   urlslug           15280 non-null  object 
 3   ID                11904 non-null  object 
 4   ALIGN             12755 non-null  object 
 5   EYE               6259 non-null   object 
 6   HAIR              11551 non-null  object 
 7   SEX               14542 non-null  object 
 8   GSM               85 non-null     object 
 9   ALIVE             15280 non-null  object 
 10  APPEARANCES       15280 non-null  float64
 11  FIRST APPEARANCE  14665 non-null  object 
 12  Year              14665 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 1.6+ MB
